In [1]:
using CSV, DataFrames, JuliaDB, Random, WiSER, vGWAS

┌ Info: Precompiling vGWAS [9514c204-b736-47c9-8157-11c3e9e5ab30]
└ @ Base loading.jl:1278


In [38]:
filepath = normpath(joinpath(dirname(pathof(WiSER)), "../data/"))
df = DataFrame!(CSV.File(filepath * "sbp.csv"))


In [39]:
vlmm = WSVarLmmModel(
    @formula(sbp ~ 1 + agegroup + gender + bmi_std + meds), 
    @formula(sbp ~ 1 + bmi_std), 
    @formula(sbp ~ 1 + agegroup + meds + bmi_std),
    :id, df);

In [40]:
WiSER.fit!(vlmm)

run = 1, ‖Δβ‖ = 0.037311, ‖Δτ‖ = 0.166678, ‖ΔL‖ = 0.100999, status = Optimal, time(s) = 0.210738
run = 2, ‖Δβ‖ = 0.005220, ‖Δτ‖ = 0.006748, ‖ΔL‖ = 0.048735, status = Optimal, time(s) = 0.228190



Within-subject variance estimation by robust regression (WiSER)
Number of individuals/clusters: 500
Total observations: 5011

Fixed-effects parameters:
───────────────────────────────────────────────────────────
                     Estimate  Std. Error       Z  Pr(>|Z|)
───────────────────────────────────────────────────────────
β1: (Intercept)   106.308       0.14384    739.07    <1e-99
β2: agegroup       14.9844      0.0633245  236.63    <1e-99
β3: gender: Male   10.0749      0.100279   100.47    <1e-99
β4: bmi_std         0.296424    0.0139071   21.31    <1e-99
β5: meds: OnMeds  -10.1107      0.122918   -82.26    <1e-99
τ1: (Intercept)    -2.5212      0.393792    -6.40    <1e-9
τ2: agegroup        1.50759     0.135456    11.13    <1e-28
τ3: meds: OnMeds   -0.435225    0.0621076   -7.01    <1e-11
τ4: bmi_std         0.0052695   0.0224039    0.24    0.8140
───────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  1.00196    

In [41]:
vlmm.data[1].storage_q◺n

3×9 Array{Float64,2}:
  0.00380916   0.00368106    0.00374837   …   0.00376479  0.00369839
 -0.0125459    0.0112974    -0.000892967     -0.00397387  0.00821902
  0.000652577  0.000487816   3.17296e-6       6.35072e-5  0.000260769

In [42]:
sum(x -> size(x.Xt, 2), vlmm.data)

5011

In [52]:
using Random
Random.seed!(333)
df[!, :rand1] = randn(size(df, 1));

In [53]:
X1vec = Matrix{Float64}[]
W1vec = Matrix{Float64}[]
for i in unique(df[!, :id])
    push!(X1vec, reshape(df[df[!, :id].==i, :rand1], :, 1))
    push!(W1vec, reshape(df[df[!, :id].==i, :rand1], :, 1))
end

In [82]:
st = vGWAS.WSVarScoreTest(vlmm, X1vec, W1vec);

In [83]:
V = 1/st.m * (transpose(st.ψ_1) * inv(st.B_11 - transpose(st.A_21) * vlmm.Ainv * st.B_21 - 
    transpose(st.B_21) * vlmm.Ainv * st.A_21 + 
    transpose(st.A_21) * vlmm.Ainv * vlmm.B * vlmm.Ainv * st.A_21) * st.ψ_1)

0.8648855615088483

In [84]:
st.ψ_1

2-element Array{Float64,1}:
 -46.87478145488263
 -56.7301877534061

In [85]:
using Distributions

In [86]:
Distributions.ccdf.(Chisq(2), V)

0.6489219828984738

In [111]:
using Random
pvals = []
for i in 1:10000
    Random.seed!(98765+i)
    df[!, :rand1] = randn(size(df, 1))
    df[!, :rand2] = randn(size(df, 1))
    X1vec = Matrix{Float64}[]
    W1vec = Matrix{Float64}[]
    for i in unique(df[!, :id])
        push!(X1vec, reshape(df[df[!, :id].==i, :rand1], :, 1))
        push!(W1vec, reshape(df[df[!, :id].==i, :rand2], :, 1))
    end
    st = vGWAS.WSVarScoreTest(vlmm, X1vec, W1vec)
    V = 1/st.m * (transpose(st.ψ_1) * inv(st.B_11 - transpose(st.A_21) * vlmm.Ainv * st.B_21 - 
        transpose(st.B_21) * vlmm.Ainv * st.A_21 + 
        transpose(st.A_21) * vlmm.Ainv * vlmm.B * vlmm.Ainv * st.A_21) * st.ψ_1)
    push!(pvals, Distributions.ccdf.(Chisq(2), V)) 
end

In [112]:
count(pvals .< 0.01)

72

In [113]:
count(pvals .< 0.05)

435

In [114]:
count(pvals .< 0.10)

948